# Complete Baseline Correction Guide for EPR Data

This comprehensive notebook demonstrates all baseline correction methods available in EPyR Tools using **real EPR data**.

## Table of Contents
1. [Introduction & Setup](#introduction)
2. [Basic Polynomial Baseline Correction](#polynomial)
3. [Stretched Exponential Correction (T2 Data)](#stretched)
4. [Bi-exponential Correction](#bi-exponential)
5. [Automatic Model Selection](#automatic)
6. [Interactive Region Selection](#interactive)
7. [2D Baseline Correction](#2d-correction)
8. [Best Practices & Tips](#best-practices)

---

## 1. Introduction & Setup {#introduction}

EPyR Tools provides a comprehensive suite of baseline correction methods specifically designed for EPR spectroscopy data:

- **Polynomial correction**: For smooth baseline drifts
- **Stretched exponential**: For T2 relaxation and echo decay measurements
- **Bi-exponential**: For complex decay patterns with multiple components
- **Automatic selection**: Intelligent model selection using statistical criteria
- **Interactive selection**: Manual region selection with matplotlib widgets


In [ ]:
import epyr
import epyr.baseline_correction as bc  # Import baseline correction functions
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import warnings

# Configure plotting
plt.style.use('default')
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 10

# Suppress minor warnings for cleaner output
warnings.filterwarnings('ignore', category=UserWarning)

print(f"EPyR Tools version: {epyr.__version__}")
print("Available data files:")
data_dir = Path("../data")
for f in sorted(data_dir.glob("*.DSC")):
    print(f"  📁 {f.name}")

## 2. Basic Polynomial Baseline Correction {#polynomial}

Polynomial correction is ideal for **continuous wave (CW) EPR spectra** with smooth baseline drifts.

In [ ]:
    # Apply polynomial baseline correction (order 3)
    print("\n🔧 Applying 3rd-order polynomial baseline correction...")
    corrected_cw, baseline_cw = bc.baseline_polynomial_1d(
        x_cw, y_cw, params_cw, 
        order=3,
        exclude_center=True,     # Exclude center region containing signal
        center_fraction=0.4      # Exclude middle 40% of data
    )

## 3. Stretched Exponential Correction (T2 Data) {#stretched}

Stretched exponential correction is perfect for **T2 relaxation measurements** and echo decay data.

The stretched exponential function is: **f(t) = A × exp(-(t/τ)^β) + offset**

- **β = 1**: Pure exponential decay
- **β < 1**: Sub-exponential (slower than exponential)
- **β > 1**: Super-exponential (faster than exponential)

In [ ]:
    # Apply stretched exponential correction
    print("\n🔧 Applying stretched exponential baseline correction...")
    corrected_t2, baseline_t2 = bc.baseline_stretched_exponential_1d(
        x_t2, y_t2, params_t2,
        beta_range=(0.01, 5.0),  # Full range for stretching parameter
        use_real_part=True,      # Fit real part of complex data
        exclude_initial=5,       # Skip first 5 points (potential artifacts)
        exclude_final=20         # Skip last 20 points
    )

## 4. Bi-exponential Correction {#bi-exponential}

Bi-exponential correction handles **complex decay patterns** with fast and slow components.

The bi-exponential function is: **f(t) = A₁×exp(-t/τ₁) + A₂×exp(-t/τ₂) + offset**

This is useful for systems with multiple relaxation pathways.

In [ ]:
    # Apply bi-exponential correction
    print("\n🔧 Applying bi-exponential baseline correction...")
    corrected_bi, baseline_bi = bc.baseline_bi_exponential_1d(
        x_bi, y_bi, params_bi,
        tau_ratio_min=2.5,     # Ensure components are sufficiently different
        use_real_part=True,
        exclude_initial=5,
        exclude_final=20
    )

## 5. Automatic Model Selection {#automatic}

The **automatic baseline selection** intelligently chooses the best model (polynomial, stretched exponential, or bi-exponential) using statistical criteria:

- **AIC** (Akaike Information Criterion): Balances fit quality vs complexity
- **BIC** (Bayesian Information Criterion): More conservative, penalizes complexity more
- **R²** (Coefficient of determination): Measures fraction of variance explained

This is perfect when you're unsure which baseline model to use!

In [ ]:
        # Run automatic model selection
        corrected, baseline, info = bc.baseline_auto_1d(
            x, y, params,
            use_real_part=True,
            exclude_initial=5,
            exclude_final=20,
            verbose=False
        )

## 6. Interactive Region Selection {#interactive}

Interactive region selection allows you to **manually specify** which parts of your data to use for baseline fitting.

**Note**: Interactive selection works best in **local Jupyter installations**. In online environments, you may need to use manual region specification.

In [ ]:
    # Apply correction with manual regions
    corrected_manual, baseline_manual = bc.baseline_polynomial_1d(
        x_demo, y_demo, params_demo,
        order=2,
        manual_regions=baseline_regions,
        region_mode='include',  # Use ONLY these regions for fitting
        exclude_center=False    # Don't auto-exclude center since we specify regions manually
    )

## 7. 2D Baseline Correction {#2d-correction}

For **2D EPR datasets** (like Rabi oscillation measurements), EPyR Tools provides specialized 2D baseline correction.

2D correction fits: **f(x,y) = Σᵢⱼ aᵢⱼ × xⁱ × yʲ**

In [ ]:
    # Apply 2D baseline correction
    print("\n🔧 Applying 2D polynomial baseline correction...")
    corrected_2d, baseline_2d = bc.baseline_polynomial_2d(
        x_2d, y_2d, params_2d,
        order=(2, 2),  # 2nd order along both axes (tuple format)
        exclude_center=True,
        center_fraction=0.3
    )

## 8. Best Practices & Tips {#best-practices}

### 🎯 **Choosing the Right Method**

| **Data Type** | **Recommended Method** | **Why** |
|---------------|------------------------|----------|
| **CW EPR spectra** | `baseline_polynomial_1d()` | Smooth baseline drifts |
| **T2 relaxation** | `baseline_stretched_exponential_1d()` | Natural exponential decay |
| **Complex decay** | `baseline_bi_exponential_1d()` | Multiple decay components |
| **Unknown baseline** | `baseline_auto_1d()` | Automatic model selection |
| **2D datasets** | `baseline_polynomial_2d()` | Surface fitting |

### 🔧 **Parameter Optimization**

In [ ]:
print("2. 🧪 METHOD SELECTION:")
print("   • CW EPR → bc.baseline_polynomial_1d(order=2-4)")
print("   • T2 data → bc.baseline_stretched_exponential_1d()")
print("   • Complex decay → bc.baseline_bi_exponential_1d()")
print("   • Unsure → bc.baseline_auto_1d() for automatic selection")
print("")

print("3. 🎛️  PARAMETER TUNING:")
print("   • Polynomial order: Start with 2-3, increase if needed")
print("   • Beta range: (0.01, 5.0) covers most physical cases")
print("   • Tau ratio: ≥2.5 for bi-exponential component separation")
print("")

print("4. 🎨 VISUALIZATION:")
print("   • Always plot original + baseline + corrected data")
print("   • Use semi-log plots for exponential analysis")
print("   • Check R² values (>0.95 typically good)")
print("")

print("5. ⚠️  COMMON PITFALLS:")
print("   • Don't over-fit: high-order polynomials can remove signal")
print("   • Exclude signal regions when fitting baseline")
print("   • For complex data, choose real part vs magnitude carefully")
print("   • Interactive selection may not work in all Jupyter environments")

# Example of good practice workflow
print("\n📝 RECOMMENDED WORKFLOW:")
print("""
# 1. Load and examine data
x, y, params, filepath = epyr.eprload("data.DSC")
plt.plot(x, np.real(y))
plt.show()""")

---

## 🎉 **Conclusion**

This notebook demonstrated the **complete baseline correction suite** in EPyR Tools using real EPR data:

1. **✅ Polynomial correction** for CW EPR spectra
2. **✅ Stretched exponential correction** for T2 relaxation data  
3. **✅ Bi-exponential correction** for complex decay patterns
4. **✅ Automatic model selection** using statistical criteria
5. **✅ Interactive and manual region selection**
6. **✅ 2D baseline correction** for advanced datasets
7. **✅ Best practices and workflow recommendations**

### **📚 Next Steps**

- Try these methods on your own EPR data
- Experiment with different parameter settings
- Use `baseline_auto_1d()` when unsure which method to choose
- Check the [EPyR documentation](https://epyr-tools.readthedocs.io) for more advanced features

### **🆘 Getting Help**

- `help(epyr.baseline_auto_1d)` - Function documentation
- `epyr.baseline_correction.jupyter_help()` - Interactive help
- Check the examples in `/examples/scripts/` for more demos

**Happy EPR data analysis! 🧲📊**